In [1]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from gcn.utils import *
from gcn.models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data('citeseer')

In [2]:
# print(type(adj),type(adj[1,:]))
# print(adj[1,:], adj.shape)

In [4]:
#features

In [5]:
features = preprocess_features(features)
support = [preprocess_adj(adj)] # Graph Laplacian from adjacency matrix , in a compressed form through sparse_to_tuple function
num_supports = 1
model_func = GCN

In [6]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [7]:
from gcn.layers import *
from gcn.metrics import *

flags = tf.app.flags
FLAGS = flags.FLAGS


class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError
        # define it in the child class as needed 

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build() # first call the child class' build

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

'''
class MLP(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(MLP, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):
        self.layers.append(Dense(input_dim=self.input_dim,
                                 output_dim=FLAGS.hidden1,
                                 placeholders=self.placeholders,
                                 act=tf.nn.relu,
                                 dropout=True,
                                 sparse_inputs=True,
                                 logging=self.logging))

        self.layers.append(Dense(input_dim=FLAGS.hidden1,
                                 output_dim=self.output_dim,
                                 placeholders=self.placeholders,
                                 act=lambda x: x,
                                 dropout=True,
                                 logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)


class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)
'''
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)
        self.learning_rate = 0.01
        self.hidden1 = 16
        self.weight_decay = 5e-4
        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)

        self.build() # from Model 

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += self.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=self.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=self.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [8]:
# Create model
model = GCN(placeholders, input_dim=features[2][1], logging=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
# Initialize session
sess = tf.Session()

In [10]:
# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

In [11]:
# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [12]:
# Train model
epochs = 20
dropout = 0.5
early_stopping = 10
for epoch in range(epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']:dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > early_stopping and cost_val[-1] > np.mean(cost_val[-(early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


Epoch: 0001 train_loss= 1.79988 train_acc= 0.12500 val_loss= 1.79539 val_acc= 0.33400 time= 0.04901
Epoch: 0002 train_loss= 1.79321 train_acc= 0.41667 val_loss= 1.79180 val_acc= 0.47000 time= 0.02750
Epoch: 0003 train_loss= 1.78746 train_acc= 0.55833 val_loss= 1.78886 val_acc= 0.50800 time= 0.02712
Epoch: 0004 train_loss= 1.78179 train_acc= 0.68333 val_loss= 1.78654 val_acc= 0.50800 time= 0.02722
Epoch: 0005 train_loss= 1.77314 train_acc= 0.73333 val_loss= 1.78472 val_acc= 0.51400 time= 0.02801
Epoch: 0006 train_loss= 1.76781 train_acc= 0.80833 val_loss= 1.78308 val_acc= 0.49800 time= 0.02734
Epoch: 0007 train_loss= 1.76107 train_acc= 0.80000 val_loss= 1.78149 val_acc= 0.49600 time= 0.02758
Epoch: 0008 train_loss= 1.75164 train_acc= 0.79167 val_loss= 1.77997 val_acc= 0.50400 time= 0.02755
Epoch: 0009 train_loss= 1.74751 train_acc= 0.77500 val_loss= 1.77841 val_acc= 0.51400 time= 0.02914
Epoch: 0010 train_loss= 1.74177 train_acc= 0.83333 val_loss= 1.77692 val_acc= 0.52000 time= 0.02799


In [13]:
model.layers[0].vars.values()[0]

<tf.Variable 'gcn/graphconvolution_1_vars/weights_0:0' shape=(3703, 16) dtype=float32_ref>

In [14]:
model.layers[1].vars.values()

[<tf.Variable 'gcn/graphconvolution_2_vars/weights_0:0' shape=(16, 6) dtype=float32_ref>]

In [15]:
print((sess.run(model.layers[0].vars.values()[0])))

[[-9.7480029e-02 -1.3641424e-01  1.8186566e-01 ...  1.3433464e-01
   1.4534225e-01  5.7039790e-02]
 [ 9.6261308e-02  1.2482219e-01 -7.5763397e-02 ... -1.1233752e-01
   1.4884260e-01  1.9499114e-01]
 [-1.1514472e-03 -5.9553361e-03 -7.6168682e-05 ...  5.2486672e-03
  -5.6344112e-03  1.0533076e-02]
 ...
 [ 7.3355916e-03 -3.9419439e-04 -2.3661207e-03 ... -7.8985374e-03
  -9.1427611e-04 -8.0452655e-03]
 [ 1.4897217e-03  7.4331206e-04 -2.8998931e-03 ... -1.7133296e-03
  -2.5036945e-03  9.5531670e-03]
 [-4.5862108e-02 -3.9389364e-02 -4.3950893e-02 ...  2.9149503e-02
  -9.5252981e-03 -2.2447281e-02]]


In [18]:
sys.version_info > (3, 0)

False

# what happens to the unlabeled data? 
everything is lebeled, they just block out some with a mask! 

In [20]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

In [44]:
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format('cora', names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)

In [46]:
x, len(y), len(y[0]), sum([1 for t in y if sum(t)==0]),tx, len(ty), len(ty[0]),sum([1 for t in ty if sum(t)==0]), allx, len(ally),len(ally[0]), sum([1 for t in ally if sum(t)!=1])

(<140x1433 sparse matrix of type '<type 'numpy.float32'>'
 	with 2647 stored elements in Compressed Sparse Row format>,
 140,
 7,
 0,
 <1000x1433 sparse matrix of type '<type 'numpy.float32'>'
 	with 17955 stored elements in Compressed Sparse Row format>,
 1000,
 7,
 0,
 <1708x1433 sparse matrix of type '<type 'numpy.float32'>'
 	with 31261 stored elements in Compressed Sparse Row format>,
 1708,
 7,
 0)

In [47]:
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format('cora'))
    test_idx_range = np.sort(test_idx_reorder)

In [56]:
    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

In [59]:
features, len(y_train), len(y_train[0])

(<2708x1433 sparse matrix of type '<type 'numpy.float32'>'
 	with 49216 stored elements in LInked List format>, 2708, 7)

In [68]:
float(len(idx_train))/features.shape[0]

0.051698670605613

In [67]:
140.0/2708

0.051698670605613

In [70]:
sum([1 for t in y_train if sum(t)!=0])

140